# RD sim on spheres/metaballs

In [ ]:
%load_ext autoreload
%autoreload 1

import os
import itertools
import colorsys

import numpy as np
from joblib import Parallel, delayed
from PIL import Image
from vapory import *

from rdmb import *
%aimport rdmb

%matplotlib inline

### sim_set
|     |     |     |
|-----|-----|-----|
| p0  | h01 | p1  |
| h02 | hmx | h13 |
| p2  | h23 | p3  |
#### p: parents, h: hybrids

In [ ]:
def sim_parent(file_base, out_dir, A, C, seed=42):
    
    np.random.seed(seed)
    mb_obj_file = "mbobj/mb_1.obj.gz"
    
    fb_p = out_dir + "/" + file_base + "_A{:.4f}_C{:.4f}_seed-{:d}".format(A, C, seed)
    rdmb.rdmb_uni_AC(mb_obj_file, fb_p, A=A, C=C, max_ite=2000)
    
    return fb_p

In [ ]:
def sim_hybrid(file_base, out_dir, As, Cs, form=0, seed=42):
    
    np.random.seed(seed)
    
    if (form==0):         # h01
        mb_obj_file = "mbobj/mb_2.obj.gz"
        fb_h = out_dir + "/" + file_base + "_h01_seed-{:d}".format(seed)
        rdmb.rdmb_grad_AC(mb_obj_file, fb_h, -1.1, 1.1,
                          As[0], As[1], Cs[0], Cs[1], max_ite=2000)

    elif (form==1):       # h23
        mb_obj_file = "mbobj/mb_2.obj.gz"
        fb_h = out_dir + "/" + file_base + "_h23_seed-{:d}".format(seed)
        rdmb.rdmb_grad_AC(mb_obj_file, fb_h, -1.1, 1.1,
                          As[2], As[3], Cs[2], Cs[3], max_ite=2000)

    elif (form==2):       # h02a
        mb_obj_file = "mbobj/mb_3.obj.gz"
        fb_h = out_dir + "/" + file_base + "_h02a_seed-{:d}".format(seed)
        rdmb.rdmb_grad_AC(mb_obj_file, fb_h, -1.5, 1.1,
                          As[0], As[2], Cs[0], Cs[2], max_ite=2000)

    elif (form==3):       # h13a
        mb_obj_file = "mbobj/mb_3.obj.gz"
        fb_h = out_dir + "/" + file_base + "_h13a_seed-{:d}".format(seed)
        rdmb.rdmb_grad_AC(mb_obj_file, fb_h, -1.6, 1.3,
                          As[3], As[1], Cs[3], Cs[1], max_ite=2000)

    elif (form==4):       # h02b
        mb_obj_file = "mbobj/mb_3.obj.gz"
        fb_h = out_dir + "/" + file_base + "_h02b_seed-{:d}".format(seed)
        rdmb.rdmb_grad_AC(mb_obj_file, fb_h, -1.6, 1.3,
                          As[2], As[0], Cs[2], Cs[0], max_ite=2000)

    elif (form==5):       # h13b
        mb_obj_file = "mbobj/mb_3.obj.gz"
        fb_h = out_dir + "/" + file_base + "_h13b_seed-{:d}".format(seed)
        rdmb.rdmb_grad_AC(mb_obj_file, fb_h, -1.5, 1.1,
                          As[1], As[3], Cs[1], Cs[3], max_ite=2000)

    elif (form==6):       # hx4a:012
        mb_obj_file = "mbobj/mb_4.obj.gz"
        fb_h = out_dir + "/" + file_base + "_hx4a_seed-{:d}".format(seed)
        pas = [(As[1] + As[2])/2.0, As[2], As[1], As[0]]
        pcs = [(Cs[1] + Cs[2])/2.0, Cs[2], Cs[1], Cs[0]]
        ccrs = [0.0, 1.0, 1.0, 1.0]
        rdmb.rdmb_blend_AC(mb_obj_file, fb_h, pas, pcs, ccrs, max_ite=2000)

    elif (form==7):       # hx4b:013
        mb_obj_file = "mbobj/mb_4.obj.gz"
        fb_h = out_dir + "/" + file_base + "_hx4b_seed-{:d}".format(seed)
        pas = [(As[0] + As[3])/2.0, As[3], As[0], As[1]]
        pcs = [(Cs[0] + Cs[3])/2.0, Cs[3], Cs[0], Cs[1]]
        ccrs = [0.0, 1.0, 1.0, 1.0]
        rdmb.rdmb_blend_AC(mb_obj_file, fb_h, pas, pcs, ccrs, max_ite=2000)

    elif (form==8):       # hx4c:123
        mb_obj_file = "mbobj/mb_4.obj.gz"
        fb_h = out_dir + "/" + file_base + "_hx4c_seed-{:d}".format(seed)
        pas = [(As[1] + As[2])/2.0, As[1], As[2], As[3]]
        pcs = [(Cs[1] + Cs[2])/2.0, Cs[1], Cs[2], Cs[3]]
        ccrs = [0.0, 1.0, 1.0, 1.0]
        rdmb.rdmb_blend_AC(mb_obj_file, fb_h, pas, pcs, ccrs, max_ite=2000)

    elif (form==9):       # hx4d:023
        mb_obj_file = "mbobj/mb_4.obj.gz"
        fb_h = out_dir + "/" + file_base + "_hx4d_seed-{:d}".format(seed)
        pas = [(As[1] + As[2])/2.0, As[0], As[3], As[2]]
        pcs = [(Cs[1] + Cs[2])/2.0, Cs[0], Cs[3], Cs[2]]
        ccrs = [0.0, 1.0, 1.0, 1.0]
        rdmb.rdmb_blend_AC(mb_obj_file, fb_h, pas, pcs, ccrs, max_ite=2000)

    elif (form==10):       # hx5a: 0123
        mb_obj_file = "mbobj/mb_5.obj.gz"
        fb_h = out_dir + "/" + file_base + "_hx5a_seed-{:d}".format(seed)
        pas = [(As[1] + As[2])/2.0, As[2], As[1], As[0], As[3]]
        pcs = [(Cs[1] + Cs[2])/2.0, Cs[2], Cs[1], Cs[0], Cs[3]]
        ccrs = [0.0, 1.0, 1.0, 1.0, 1.0]
        rdmb.rdmb_blend_AC(mb_obj_file, fb_h, pas, pcs, ccrs, max_ite=2000)

    elif (form==11):       # hx5b: 0123
        mb_obj_file = "mbobj/mb_5.obj.gz"
        fb_h = out_dir + "/" + file_base + "_hx5b_seed-{:d}".format(seed)
        pas = [(As[0] + As[3])/2.0, As[3], As[0], As[1], As[2]]
        pcs = [(Cs[0] + Cs[3])/2.0, Cs[3], Cs[0], Cs[1], Cs[2]]
        ccrs = [0.0, 1.0, 1.0, 1.0, 1.0]
        rdmb.rdmb_blend_AC(mb_obj_file, fb_h, pas, pcs, ccrs, max_ite=2000)

    return fb_h

In [ ]:
AC_sets = [
    [[0.08, 0.08, 0.08, 0.08], [0.01, 0.06, 0.27, 0.307]],
    [[0.07, 0.075, 0.116, 0.12], [0.08, 0.08, 0.08, 0.08]],
    [[0.075, 0.08, 0.08, 0.12], [0.08, 0.01, 0.27, 0.08]],  
]

In [ ]:
As = AC_sets[0][0]
Cs = AC_sets[0][1]

display(As, Cs)

In [ ]:
%%time

np.random.seed(42)

out_dir = "rd_out/set1"

try:
    os.makedirs("./"+out_dir)
except FileExistsError:
    pass

fb_ps = Parallel(n_jobs=4, verbose=1, max_nbytes=None)(
            [delayed(sim_parent)(file_base="ps",
                                 out_dir=out_dir,
                                 A=AC[0], C=AC[1],
                                 seed=np.random.randint(10000))
             for AC in zip(As, Cs)])

fb_hs = Parallel(n_jobs=12, verbose=1, max_nbytes=None)(
            [delayed(sim_hybrid)(file_base="hs",
                                 out_dir=out_dir,
                                 As=As, Cs=Cs,
                                 form=i, seed=np.random.randint(10000))
             for i in range(12)])

In [ ]:
pngfname = rdmb.rdmb_povray(fb_hs[6], 2000, width=600, height=400, angle=11)
display(Image.open(pngfname))

In [ ]:
def sim_set_scene(fb_ps, fb_hs, form=0, mode="C"):
    
    vs_p0, ucs_p0, A_p0, C_p0 = rdmb.load_rd_mb(fb_ps[0])
    vs_p1, ucs_p1, A_p1, C_p1 = rdmb.load_rd_mb(fb_ps[1])
    vs_p2, ucs_p2, A_p2, C_p2 = rdmb.load_rd_mb(fb_ps[2])
    vs_p3, ucs_p3, A_p3, C_p3 = rdmb.load_rd_mb(fb_ps[3])
    
    vs_h01, ucs_h01, A_h01, C_h01 = rdmb.load_rd_mb(fb_hs[0])
    vs_h23, ucs_h23, A_h23, C_h23 = rdmb.load_rd_mb(fb_hs[1])
    
    if (form==0):
        vs_h02, ucs_h02, A_h02, C_h02 = rdmb.load_rd_mb(fb_hs[2])
        vs_h13, ucs_h13, A_h13, C_h13 = rdmb.load_rd_mb(fb_hs[3])
        vs_hmx, ucs_hmx, A_hmx, C_hmx = rdmb.load_rd_mb(fb_hs[6])
        h02_rotz = -90
        h13_rotz = 90
        hmx_rotz = 45
        rotx = 0
        
    elif (form==1):
        vs_h02, ucs_h02, A_h02, C_h02 = rdmb.load_rd_mb(fb_hs[4])
        vs_h13, ucs_h13, A_h13, C_h13 = rdmb.load_rd_mb(fb_hs[5])
        vs_hmx, ucs_hmx, A_hmx, C_hmx = rdmb.load_rd_mb(fb_hs[7])
        h02_rotz = 90
        h13_rotz = -90
        hmx_rotz = 135
        rotx = 180
        
    elif (form==2):
        vs_h02, ucs_h02, A_h02, C_h02 = rdmb.load_rd_mb(fb_hs[2])
        vs_h13, ucs_h13, A_h13, C_h13 = rdmb.load_rd_mb(fb_hs[3])
        vs_hmx, ucs_hmx, A_hmx, C_hmx = rdmb.load_rd_mb(fb_hs[8])
        h02_rotz = -90
        h13_rotz = 90
        hmx_rotz = -135
        rotx = 0
        
    elif (form==3):
        vs_h02, ucs_h02, A_h02, C_h02 = rdmb.load_rd_mb(fb_hs[4])
        vs_h13, ucs_h13, A_h13, C_h13 = rdmb.load_rd_mb(fb_hs[5])
        vs_hmx, ucs_hmx, A_hmx, C_hmx = rdmb.load_rd_mb(fb_hs[9])
        h02_rotz = 90
        h13_rotz = -90
        hmx_rotz = -45
        rotx = 180
        
    elif (form==4):
        vs_h02, ucs_h02, A_h02, C_h02 = rdmb.load_rd_mb(fb_hs[2])
        vs_h13, ucs_h13, A_h13, C_h13 = rdmb.load_rd_mb(fb_hs[3])
        vs_hmx, ucs_hmx, A_hmx, C_hmx = rdmb.load_rd_mb(fb_hs[10])
        h02_rotz = -90
        h13_rotz = 90
        hmx_rotz = 45
        rotx = 0
        
    elif (form==5):
        vs_h02, ucs_h02, A_h02, C_h02 = rdmb.load_rd_mb(fb_hs[4])
        vs_h13, ucs_h13, A_h13, C_h13 = rdmb.load_rd_mb(fb_hs[5])
        vs_hmx, ucs_hmx, A_hmx, C_hmx = rdmb.load_rd_mb(fb_hs[11])
        h02_rotz = 90
        h13_rotz = -90
        hmx_rotz = 135
        rotx = 180
        
    camera = Camera('location', [0, 0, -45],
                    'look_at', [0, 0, 0],
                    'angle', 12,
                    'right x*image_width/image_height')
    
    light = LightSource([-10, 10, -20],
                        'color', [1.0, 1.0, 1.0], 'parallel')
    light2 = LightSource([10, -6, -20],
                         'color', [0.2, 0.2, 0.2], 'parallel')
    background = Background('color', [1, 1, 1, 1])

    spheres = []
    xd = 3.0
    # xdd = 0.1
    xdd = 0.2

    spheres += rdmb.sph(vs_p0, ucs_p0,
                        np.ones(len(vs_p0))*A_p0,
                        np.ones(len(vs_p0))*C_p0,
                        -xd, xd, 0, rotx=form*60, mode=mode)
    spheres += rdmb.sph(vs_p1, ucs_p1,
                        np.ones(len(vs_p1))*A_p1,
                        np.ones(len(vs_p1))*C_p1,
                        xd, xd, 0, rotx=form*60, mode=mode)
    spheres += rdmb.sph(vs_p2, ucs_p2,
                        np.ones(len(vs_p2))*A_p2,
                        np.ones(len(vs_p2))*C_p2,
                        -xd, -xd, 0, rotx=form*60, mode=mode)
    spheres += rdmb.sph(vs_p3, ucs_p3,
                        np.ones(len(vs_p3))*A_p3,
                        np.ones(len(vs_p3))*C_p3,
                        xd, -xd, 0, rotx=form*60, mode=mode)
    
    spheres += rdmb.sph(vs_h01, ucs_h01,
                        A_h01, C_h01,
                        0, xd, 0, rotx=form*60, mode=mode)
    spheres += rdmb.sph(vs_h23, ucs_h23,
                        A_h23, C_h23,
                        0, -xd, 0, rotx=form*60, mode=mode)
    
    spheres += rdmb.sph(vs_h02, ucs_h02,
                        A_h02, C_h02,
                        -xd+xdd, 0, 0,
                        rotx=rotx, rotz=h02_rotz, mode=mode)
    spheres += rdmb.sph(vs_hmx, ucs_hmx,
                        A_hmx, C_hmx,
                        0, 0, 0, rotx=rotx, rotz=hmx_rotz, mode=mode)
    spheres += rdmb.sph(vs_h13, ucs_h13,
                        A_h13, C_h13,
                        xd-xdd, 0, 0, rotx=rotx, rotz=h13_rotz, mode=mode)
    
    objects = [light, light2, background] + spheres

    scene = Scene(camera, objects=objects)
    
    return scene

In [ ]:
%%time

scenes = [sim_set_scene(fb_ps=fb_ps,
                        fb_hs=fb_hs,
                        form=i,
                        mode="C") for i in range(1)]

In [ ]:
png_file = "rdmb_set1_0.png"
scenes[0].render(png_file,
                 width=800,
                 height=800,
                 output_alpha=True,
                 antialiasing=0.001)
display(Image.open(png_file))

In [ ]:
png_file = "rdmb_set1_0_hires.png"
scenes[0].render(png_file,
                 width=2400,
                 height=2400,
                 output_alpha=True,
                 antialiasing=0.001)